In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
# Load image
image_path = '../data/fishing_boat.bmp'
image = Image.open(image_path).convert('L')

# Convert the image to a numpy array
image_array = np.array(image)